# L0 data clean

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import imp
import datetime
import pytz
%matplotlib notebook

In [2]:
#import self-written libraries
import LVL1
import settings
Glacier=settings.Glacier
Station=settings.Station
base_path=settings.base_path
new_data=True

## Define Glacier and Station

In [3]:
data_columns, general_data_columns, out_date_format, precip_columns, precip_gage_change, primary_temp_column, temp_columns, timezone, wind_col, wind_dir_columns = settings.get_settings(settings.Glacier, settings.Station)

print(Glacier)
print(Station)
print(base_path)

Gulkana
1480
C:\Users\ehbaker\Documents/BrokenServer/


In [4]:
##Uncomment this if in .py not jupyter notebook
# if (Glacier =='Sperry') | (Glacier=='SouthCascade'):
#     import sys
#     print('start at L1 data for Sperry; L0 data provided from MT and made with "Data\Sperry\AllYears\Wx\Raw\CreateL0Data_FromSperryFormat.py"')
#     sys.exit()

Logger Data

In [5]:
base_path + 'Code/Weather/LVL_0/ConcatenateRawWxData.py'

'C:\\Users\\ehbaker\\Documents/BrokenServer/Code/Weather/LVL_0/ConcatenateRawWxData.py'

In [6]:
#This combines raw logger files into a single data file for all logger downloads @ a Campbell station
if new_data:
    add_new_logger_data_script_pth='"' + base_path + 'Code/Weather/LVL_0/ConcatenateRawWxData.py'+ '"'
    %run $add_new_logger_data_script_pth
    if Glacier + Station in['Wolverine1420', 'Gulkana1920', 'SouthCascade1830', 'JuneauIcefieldCamp17AWS', 'JuneauIcefieldCamp18AWS', 'JuneauIcefieldCamp10AWS']: 
        jws_add_new_data_pth='"' + base_path + 'Code/Weather/LVL_0/ConcatenateRawCampbellWxData.py'+ '"'
        %run $jws_add_new_data_pth

 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2010\Wx\gulkana1480_2010_0226.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2010\Wx\gulkana1480_2010_0301.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2010\Wx\gulkana1480_2010_0306.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2010\Wx\gulkana1480_2010_1021.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2011\Wx\gulkana1480_2011_0306_edit_RecreatedFromExcel.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2011\Wx\gulkana1480_2011_0629.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2011\Wx\gulkana1480_2011_0701.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2011\Wx\gulkana1480_2011_0824.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana\2011\Wx\gulkana1480_2011_0825.txt
 concatinating C:\Users\ehbaker\Documents/BrokenServer/Dat

In [7]:
if (Glacier + Station =='Wolverine370'):
    print ('STOP: not needed here; start at L1. All data from NWIS')
    print('Ensure that Add1990sDataFromNWIS.py has been run')
    if new_data==True:
        add1990sNWISpth='Add1990sDataFromNWIS'
        %run $add1990sNWISpth
    sys.exit() #uncomment for non-ipynb script

In [8]:
#Get output into 15-minute interval
if (Glacier + Station== 'Wolverine1420') | (Glacier + Station== 'Gulkana1920'): #If a JWS station, which logs @ 5 minutes
    if new_data:
        convert5minto15min_script_pth='"' + base_path + 'Code/Weather/LVL_0/ConvertFrom5minTo15minData_JWS.py'+ '"'
        %run $convert5minto15min_script_pth

In [9]:
#Combine logger and telemetered data, convert to 15-minute timestep
if Glacier + Station in ['JuneauIcefieldCamp17AWS', 'JuneauIcefieldCamp18AWS', 'JuneauIcefieldCamp10AWS', 'SouthCascade1830']: 
    if new_data:
        converthourlyminto15min_script_pth='"' + base_path + 'Code/Weather/LVL_0/CombineTelemeteredAndLoggerData_SouthCascadeAndJuneauIcefield.py'+ '"'
        %run $converthourlyminto15min_script_pth

In [10]:
if ('JuneauIcefield' in Glacier and 'AWS' not in Station and new_data): #for JIRP hobo logger stations, and South Cascade Hut:
    converthourlyto15minJIRPpth='"' + base_path +r"Code\Weather\LVL_0\ConvertFromhourlyTo15minDat.py" +'"'
    %run $converthourlyto15minJIRPpth
    #sys.exit() #uncomment for non-ipynb script

In [11]:
#At the Wolverine high JWS site, add TPG (was logged internally, never wired in to logger direclty before moving).
if (Glacier + Station== 'Wolverine1420'): #add TPG log to the high wolverine station, during the time which it existed there.
    addTPGdata_script_pth='"' + base_path + 'Code/Weather/LVL_0/AddWolverineTPGData.py'+ '"'
    
    %run $addTPGdata_script_pth

In [12]:
#Read in data
folder=base_path+ r"Data/" + Glacier+ "/AllYears/Wx/Raw" #folder cont
file_label='_15min_all' #used for importting and output name
file=Glacier.lower()+Station+file_label+".csv" #filename; change if yours different
pth=os.path.join(folder, file) #path to data file

In [13]:
#Set constants for input data
date_format='%m/%d/%Y %H:%M:%S' #input date format (change if yours is different)

In [14]:
dat=pd.read_csv(pth)
print("read file: " + pth)
dat.tail(10)

read file: C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana/AllYears/Wx/Raw\gulkana1480_15min_all.csv


,Date,Time,Instrument,Value,Unit,Flag,DateTime
3522848,08/22/2020,14:00:00,AT2,11.600,C,G,08/22/2020 14:00:00
3522849,08/22/2020,14:15:00,WSG,9.100,m/s,G,08/22/2020 14:15:00
3522850,08/22/2020,14:15:00,WD,279.000,degrees,G,08/22/2020 14:15:00
3522851,08/22/2020,14:15:00,T,11.700,C,G,08/22/2020 14:15:00
3522852,08/22/2020,14:15:00,AT,11.700,C,G,08/22/2020 14:15:00
3522853,08/22/2020,14:15:00,WS,5.100,m/s,G,08/22/2020 14:15:00
3522854,08/22/2020,14:15:00,BV,13.700,V,G,08/22/2020 14:15:00
3522855,08/22/2020,14:15:00,Tinternal,14.100,C,G,08/22/2020 14:15:00
3522856,08/22/2020,14:15:00,PC,0.479,m,G,08/22/2020 14:15:00
3522857,08/22/2020,14:15:00,AT2,11.900,C,G,08/22/2020 14:15:00


In [15]:
dat.loc[:,'DateTime']=pd.to_datetime(dat['DateTime'], infer_datetime_format=True) #set to date-time from string

## Convert Logger time to UTC and Local Time

In [16]:
#Correct for time slips
dat['DateTime']=dat['DateTime'].dt.round('15min') #round time to the nearest 15 minute value

#Convert to UTC time, from logger time
if (Glacier+Station=='Wolverine990') | (Glacier+Station=='Gulkana1480') | (Glacier+Station=='Gulkana1920') | (Glacier+Station=='Wolverine1420'):
    #dat['UTC_col']=dat.DateTime + datetime.timedelta(hours=8)
    dat['UTC_col']=[x + datetime.timedelta(hours=8) for x in dat.DateTime]
if (Glacier+Station=='Gulkana1725'):
    dat['UTC_col']=dat.DateTime #logger is already in UTC time
if Glacier + Station=='Sperry2440':
    dat['UTC_col']=dat.UTC_time.copy()
if 'AWS' in Station:
    dat['UTC_col']=dat.DateTime + datetime.timedelta(hours=9) #GMT -9 for Juneau
if Glacier+Station=='SouthCascade1830':
    dat['UTC_col']=dat.DateTime #figured out in pre-processing of combining telmetry and logger data

In [17]:
dat['UTC_col']=pd.to_datetime(dat['UTC_col'], format=date_format)

In [18]:
#Set timezone of known UTC column
dat['UTC_col'].timezone='UTC'
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

In [19]:
dat=dat.set_index('UTC_col')
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

#Create local time
dat['local_time'] = dat.index.tz_localize('UTC').tz_convert(local_timezone)

dat['index_local_time']=dat.local_time.copy()

#Set index to local time
dat=dat.set_index('index_local_time')

In [20]:
#Drop duplicates that have same instrument name and date/time stamp (from multiple downloads of same portion of data)
dat=dat.drop_duplicates(subset=['DateTime', 'Instrument'], keep='first') #gets rid of overlaps (i.e. multiple downloads of same bit of data)

In [21]:
dat.tail(10)

,Date,Time,Instrument,Value,Unit,Flag,DateTime,local_time
index_local_time,,,,,,,,
2020-08-22 14:00:00-08:00,08/22/2020,14:00:00,AT2,11.600,C,G,2020-08-22 14:00:00,2020-08-22 14:00:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,WSG,9.100,m/s,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,WD,279.000,degrees,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,T,11.700,C,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,AT,11.700,C,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,WS,5.100,m/s,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,BV,13.700,V,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,Tinternal,14.100,C,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,PC,0.479,m,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00


In [22]:
#Instrument Label Change - if same label has been used for different quantities thru time: separate into 2 distinct labels.
if Glacier + Station=="Wolverine990":
    dat.loc[(dat.Instrument=='WD') & (dat.index<'2015-09-06 17:15'), 'Instrument']='WindDir' #before this date, "WD" was simply wind direction; after, is vector-averaged
    dat.loc[(dat.Instrument=='WD') & (dat.index>='2015-09-06 17:15'), 'Instrument']='WDSA' 
    
if Glacier + Station=="Gulkana1480":
    dat.loc[(dat.Instrument=='WD') & (dat.index<'2015-09-28 12:00'), 'Instrument']='WindDir' #before this date, "WD" was simply wind direction; after, is vector-averaged
    dat.loc[(dat.Instrument=='WD') & (dat.index>='2015-09-28 12:00'), 'Instrument']='WDSA' 
    
#Gulkana 1725 has had consistent variable naming throughout

## Consolidate Sensor Naming
#### Has varied thru time

In [23]:
#If site does NOT have a given sensor, leave label, create empty list

if Glacier + Station =='Wolverine990':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WindGusts', 'WG', 'WSG']
    wind_speed_labels=['AvgWind', 'WS'] #this is NOT vector-averaged wind speed
    vec_avg_wind_dir=['WDSA']
    wind_dir_labels=['WindDir']
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AirTempAsp', 'AspT', 'AT']
    temp_asp2_labels=['ASPTRH', 'AT2']
    precip_stage_labels=['PR']
    precip_weighing_labels=['TPG', 'PC']
    precip_incremental_labels=[]
    rel_hum_labels=['RH']
    barometer_labels=['BAROMETER', 'B']
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'AirTemp3', 'Air Temp w/FanN', 'AirTemp w/Fan', 'VectorAvgWindDir', 'PS', 'MWD', 'WDA'] #labels containing only bad data; introduced during maintenence etc.
    radiation_up_labels=[]
    radiation_down_labels=[]
    snow_depth_labels=[] 
    
if Glacier + Station =='Gulkana1480':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WSG']
    wind_speed_labels=['WindSpeed', 'WS']
    vec_avg_wind_dir=['WDSA']
    wind_dir_labels=['WindDir'] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AspT', 'AT']
    temp_asp2_labels=['AT2']
    precip_stage_labels=['PR'] #precip doesn't begin until 2010-07; before this, is recording, but logger only has 0 precip
    precip_weighing_labels=['TPG', 'PC']
    precip_incremental_labels=[]
    rel_hum_labels=['RH']
    barometer_labels=[] #no barometer at this station
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'AirTemp3', 'Stage', 'AirTempAsp', 'AirTemp w/Fan'] #labels containing only bad data; introduced during maintenence etc.
    radiation_up_labels=[]
    radiation_down_labels=[]
    snow_depth_labels=[]    
    
if Glacier + Station=='Gulkana1725':
    #Only a single name thru time; no changes @ 1725
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['MS']
    vec_avg_wind_dir=['MD']
    wind_dir_labels=[] #this is NOT-vector average wind direction
    temp_unasp1_labels=['TA'] #TA is NOT Aspriated; it's Temperature, Air
    temp_unasp2_labels=['T']
    temp_unasp3_labels=['T3'] #this is the only site with 3 un-aspirated temperature sensors!!!
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=['PC']
    precip_incremental_labels=[]
    rel_hum_labels=[]
    barometer_labels=[] #no barometer at this station
    logger_bat_label=['VB']
    logger_temp_label=[]
    bad_labels=[]
    radiation_up_labels=['RU']
    radiation_down_labels=['RD']
    snow_depth_labels=['DS']
    
if (Glacier + Station=='Gulkana1920') | (Glacier + Station=='Wolverine1420'): #same program and same variable naming at JWS stations
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['WS_ms_S_WVT']
    vec_avg_wind_dir=['WindDir_D1_WVT'] #I am guessing this is vec acerage; should come back and check program before using
    wind_dir_labels=[] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTempC_Avg']
    temp_unasp2_labels=[]
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=[]
    precip_incremental_labels=[]
    if Station=='Wolverine1420':#only true at Wolverine
        precip_weighing_labels=['TPGCumulative']
    rel_hum_labels=['RH']
    barometer_labels=['BP_inHg_Avg']
    logger_bat_label=['BattV_Avg']
    logger_temp_label=[]
    bad_labels=[]
    radiation_up_labels=['PyrnUp_Avg']
    radiation_down_labels=['PyrnDn_Avg']
    snow_depth_labels=[]

    
if Glacier =='JuneauIcefield' and "AWS" in Station: #differentiate between 3 AWS stations, vs. jirp hobo loggers
    wind_gust_labels=['WindSpMax', 'WindSpGust', 'WindSP_Gust']#no wind gust recorded
    wind_speed_labels=['WS_ms_S_WVT', 'WindSpeed', 'WindSp', 'WindSP']
    vec_avg_wind_dir=['WindDir_D1_WVT', 'WindDir_VecAvg'] #I am guessing this is vec acerage; should come back and check program before using
    wind_dir_labels=['WindDir'] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTC_Avg', 'AirTemp']
    temp_unasp2_labels=[]
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=[]
    precip_incremental_labels=['Precip_m']
    rel_hum_labels=['RH']
    barometer_labels=['BP_mbar', 'BP_kPa_Avg', 'Baro_Kpa', 'Baro']
    logger_bat_label=['BattV_Avg']
    logger_temp_label=[]
    bad_labels=['Time']
    radiation_up_labels=['PyrnUp_Avg', 'SolRad']
    radiation_down_labels=['PyrnDn_Avg']
    snow_depth_labels=['TCDT', 'SnowDep1']

if Glacier + Station=='SouthCascade1830':
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['WS_ms_AVG', 'WindSpeed']
    vec_avg_wind_dir=[]
    wind_dir_labels=['WindDir'] #this is NOT-vector average wind direction
    temp_unasp1_labels=['T107_C_AVG', 'Tpassive1'] #TA is NOT Aspriated; it's Temperature, Air
    temp_unasp2_labels=['RH_AirTC_AVG']
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=[] #has some tipping bucket info, but will need a lot of work before could be salvagable
    precip_incremental_labels=[]
    rel_hum_labels=['RelHum', 'RH']
    barometer_labels=[] #no barometer at this station
    logger_bat_label=[]
    logger_temp_label=[]
    bad_labels=['Precip', 'TipBucketCumulative', 'RECORD']
    radiation_up_labels=['RadiationIn']
    radiation_down_labels=[]
    snow_depth_labels=['DS']

In [24]:
#Rename instruments 
dat.loc[dat.Instrument.isin(wind_gust_labels), "Instrument"]='WindGustSpeed'
dat.loc[dat.Instrument.isin(wind_speed_labels), "Instrument"]='WindSpeed'
dat.loc[dat.Instrument.isin(wind_dir_labels), "Instrument"]='WindDir'
dat.loc[dat.Instrument.isin(vec_avg_wind_dir), "Instrument"]='VecAvgWindDir'
dat.loc[dat.Instrument.isin(temp_unasp1_labels), "Instrument"]='Tpassive1'
dat.loc[dat.Instrument.isin(temp_unasp2_labels), "Instrument"]='Tpassive2'
dat.loc[dat.Instrument.isin(temp_asp1_labels), "Instrument"]='TAspirated1'
dat.loc[dat.Instrument.isin(temp_asp2_labels), "Instrument"]='TAspirated2'
if 'temp_unasp3_labels' in locals():
    dat.loc[dat.Instrument.isin(temp_unasp3_labels), "Instrument"]='Tpassive3'
dat.loc[dat.Instrument.isin(precip_stage_labels), "Instrument"]='StageCumulative'
dat.loc[dat.Instrument.isin(precip_weighing_labels), "Instrument"]='TPGCumulative'
dat.loc[dat.Instrument.isin(precip_incremental_labels), "Instrument"]='IncrementalPrecip'
dat.loc[dat.Instrument.isin(rel_hum_labels), "Instrument"]='RelHum'
dat.loc[dat.Instrument.isin(barometer_labels), "Instrument"]='Barom'
dat.loc[dat.Instrument.isin(logger_bat_label), "Instrument"]='LoggerBattery'
dat.loc[dat.Instrument.isin(logger_temp_label), "Instrument"]='LoggerTemp'
dat.loc[dat.Instrument.isin(radiation_up_labels), "Instrument"]='RadiationIn'
dat.loc[dat.Instrument.isin(radiation_down_labels), "Instrument"]='RadiationOut'
dat.loc[dat.Instrument.isin(snow_depth_labels), "Instrument"]='SnowDepth'

#Remove rows with bad data
dat=dat[~dat.Instrument.isin(bad_labels)] #Remove where labels are bad; introduced during maintenance

In [25]:
print(dat.Instrument.unique())
dat.tail(10)

['WindDir' 'WindSpeed' 'Tpassive1' 'Tpassive2' 'TAspirated1'
 'StageCumulative' 'LoggerBattery' 'LoggerTemp' 'WindGustSpeed'
 'VecAvgWindDir' 'TPGCumulative' 'TAspirated2' 'RelHum']


,Date,Time,Instrument,Value,Unit,Flag,DateTime,local_time
index_local_time,,,,,,,,
2020-08-22 14:00:00-08:00,08/22/2020,14:00:00,TAspirated2,11.600,C,G,2020-08-22 14:00:00,2020-08-22 14:00:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,WindGustSpeed,9.100,m/s,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,VecAvgWindDir,279.000,degrees,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,Tpassive1,11.700,C,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,TAspirated1,11.700,C,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,WindSpeed,5.100,m/s,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,LoggerBattery,13.700,V,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,LoggerTemp,14.100,C,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00
2020-08-22 14:15:00-08:00,08/22/2020,14:15:00,TPGCumulative,0.479,m,G,2020-08-22 14:15:00,2020-08-22 14:15:00-08:00


In [26]:
not_used_cols=set(data_columns) -set(dat.Instrument.unique())
if len(not_used_cols)>0:
    print (str(not_used_cols) + "not used")

In [27]:
#Cast the data from long -> wide format
#dat=dat.reset_index() #reset index so all unique
wide_dat=dat.pivot_table(columns='Instrument', values='Value', index='local_time')

In [28]:
wide_dat.tail(3)

Instrument,LoggerBattery,LoggerTemp,RelHum,StageCumulative,TAspirated1,TAspirated2,TPGCumulative,Tpassive1,Tpassive2,VecAvgWindDir,WindDir,WindGustSpeed,WindSpeed
local_time,,,,,,,,,,,,,
2020-08-22 13:45:00-08:00,13.7,13.8,NaN,NaN,10.6,10.9,0.488,11.4,NaN,279.0,NaN,5.1,2.6
2020-08-22 14:00:00-08:00,13.7,13.8,NaN,NaN,11.3,11.6,0.488,11.5,NaN,279.0,NaN,5.9,2.3
2020-08-22 14:15:00-08:00,13.7,14.1,NaN,NaN,11.7,11.9,0.479,11.7,NaN,279.0,NaN,9.1,5.1


In [29]:
#Reindex to 15min to ensure no timesteps are skipped
wide_dat.sort_index(inplace=True)
full_range_15_min = pd.date_range(wide_dat.index[0], wide_dat.index[-1], freq='15min')

In [30]:
wide_dat=wide_dat.reindex(index=full_range_15_min, fill_value=pd.np.nan)

In [31]:
#Reimport the out_date_format; getting changed at 1420 fro some reason
data_columns, general_data_columns, out_date_format, precip_columns, precip_gage_change, primary_temp_column, temp_columns, timezone, wind_col, wind_dir_columns = settings.get_settings(settings.Glacier, settings.Station)

In [32]:
#Set output format of time
wide_dat['UTC_time']=wide_dat.index.tz_convert('UTC').strftime(out_date_format)
wide_dat['local_time']=wide_dat.index.tz_convert(timezone).strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)

In [33]:
save_dat=wide_dat

In [34]:
save_dat.tail(1)

Instrument,LoggerBattery,LoggerTemp,RelHum,StageCumulative,TAspirated1,TAspirated2,TPGCumulative,Tpassive1,Tpassive2,VecAvgWindDir,WindDir,WindGustSpeed,WindSpeed,UTC_time,local_time
2020-08-22 14:15:00-08:00,13.7,14.1,NaN,NaN,11.7,11.9,0.479,11.7,NaN,279.0,NaN,9.1,5.1,2020/08/22 22:15,2020/08/22 14:15


** Adjust for size of stage gage orifice (x5) **
- this is a legacy issue; gage orifice used is bigger than instrument design. Thus, precip recorded on logger is 1/5 of true

In [35]:
precip_present=len(precip_columns)>0
if precip_present:
    if 'StageCumulative' in save_dat.columns:
        print('adjusted legacy data')
        save_dat['StageCumulative']=save_dat.StageCumulative*5 #legacy issue; see comment above

adjusted legacy data


In [36]:
#directory to save output data
save_pth=base_path+ r"Data/" +Glacier+ r"/AllYears/Wx/LVL0/"
out_nm=Glacier.lower()+ Station + file_label+ "_LVL0.csv"
out_nm

'gulkana1480_15min_all_LVL0.csv'

In [37]:
for col in not_used_cols:
    save_dat[col]=pd.np.nan

In [38]:
save_dat[['local_time', 'UTC_time'] +data_columns].to_csv(os.path.join(save_pth, out_nm), index=False, float_format='%g')
print( "saved to: ", save_pth+out_nm)

saved to:  C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana/AllYears/Wx/LVL0/gulkana1480_15min_all_LVL0.csv


In [39]:
#If this is the Nunatak station, fill gaps with Irridium telemetry
if Glacier + Station=='Gulkana1725':
    %run "./Gulkana1725_FillGapsWithIrridium.py"

In [40]:
save_dat.tail(1)

Instrument,LoggerBattery,LoggerTemp,RelHum,StageCumulative,TAspirated1,TAspirated2,TPGCumulative,Tpassive1,Tpassive2,VecAvgWindDir,WindDir,WindGustSpeed,WindSpeed,UTC_time,local_time
2020-08-22 14:15:00-08:00,13.7,14.1,NaN,NaN,11.7,11.9,0.479,11.7,NaN,279.0,NaN,9.1,5.1,2020/08/22 22:15,2020/08/22 14:15


In [41]:
#If Wolv 990 or Gulkana 1480, add 1990s data from NWIS to extend record back in time
if (Glacier + Station=='Wolverine990') | (Glacier + Station=='Gulkana1480'):    
    %run "./Add1990sDataFromNWIS.py" #this combines logger and NWIS data, from all eras not just 1990s

adjusted NWIS legacy data by multiplying by 5


C:\Users\ehbaker\Documents\Python\WXmunge\Add1990sDataFromNWIS.py:99: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  AllData=pd.concat([NWIS_data_to_add, dat], axis=0)


1990s data added to logger from NWIS at Gulkana1480
wrote data to C:\Users\ehbaker\Documents/BrokenServer/Data/Gulkana/AllYears/Wx/LVL0/telemetry_added/gulkana1480_15min_all_LVL0.csv


In [42]:
pwd

'C:\\Users\\ehbaker\\Documents\\Python\\WXmunge'

In [43]:
data_columns

['Tpassive1',
 'Tpassive2',
 'TAspirated1',
 'TAspirated2',
 'TPGCumulative',
 'StageCumulative',
 'WindSpeed',
 'WindDir',
 'VecAvgWindDir',
 'RelHum',
 'WindGustSpeed']